In [1]:
import os
import json
from utils_embeddings import get_embedding_across_layers_audio, get_embeddings_audio, save_json_with_embedding
import sys
sys.path.insert(0, '../../layer-analysis')
from easyaudio.easyaudio.hub import get_model


/home/cbolanos/miniconda3/envs/tesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cbolanos/layer-analysis/embeddings/../../layer-analysis/easyaudio/easyaudio/models/byola/byol_a/common.py:31: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")


In [12]:
import gin

gin.enter_interactive_mode()

BEATs info:

Following [Gong et al., 2021a, 2022a], we convert the sample rate of each raw
waveform to 16,000, and extract the 128-dimensional Mel-filter bank features with a 25ms Povey
window that shifts every 10 ms as the acoustic feature. The acoustic feature is normalized to the mean
value of 0 and standard deviation of 0.5 following the previous works. We split each acoustic feature
into the 16 × 16 patches, and further flat them to the patch sequence as the input of our BEAT S
tokenizers and models.

In [39]:
model = get_model('byola_2048')

audio = "../../datasets/librispeech-clean-16k/dev-clean/84/121123/84-121123-0006"
audio_path = os.path.join(audio + '.flac')
reps = get_embeddings_audio(audio_path, model, 'cuda')


 2024/05/1 > 16:20:19 INFO Loading byola_2048 


In [40]:
reps

[array([[[[1.59532654e+00, 1.08367324e+00, 8.70211840e-01, ...,
           3.09083909e-01, 5.99813163e-01, 0.00000000e+00],
          [7.43736386e-01, 1.09170413e+00, 1.36931670e+00, ...,
           1.01318145e+00, 7.76527643e-01, 0.00000000e+00],
          [1.50381589e+00, 1.33462846e+00, 1.14469075e+00, ...,
           1.64319241e+00, 1.55410111e+00, 8.14249218e-01],
          ...,
          [9.64076459e-01, 8.81094992e-01, 9.72174644e-01, ...,
           8.90532970e-01, 8.18096519e-01, 1.57571709e+00],
          [8.60268533e-01, 9.71468866e-01, 1.10796535e+00, ...,
           9.68694866e-01, 1.00413346e+00, 1.78663814e+00],
          [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
           0.00000000e+00, 0.00000000e+00, 3.63729328e-01]],
 
         [[5.29685736e-01, 0.00000000e+00, 0.00000000e+00, ...,
           0.00000000e+00, 4.29154634e-01, 2.35442162e+00],
          [9.57746625e-01, 0.00000000e+00, 2.75044501e-01, ...,
           1.37628868e-01, 4.08902586e-01, 7.84608

In [66]:
len(reps[0][0][63][32])

563

In [79]:
reps[5][0]

array([75.33375 ,  0.      , 30.953926, ..., 58.512733,  0.      ,
        0.      ], dtype=float32)

In [80]:
len(reps[5][0])


2048

reps[i][0][j][k]

In [ ]:

model = get_model('BEATs_iter3')

# For every audio, process each word and save its averaged embeddings   
for audio, items in tqdm(data.items(), desc="Processing audio files"):
    embeddings_audio = {}
    audio_path = os.path.join(audio + '.flac')
    reps = get_embeddings_audio(audio_path, model, args.device)
    for word, intervals in items.items():
        if word != 'transcript_audio':
            embeddings_list = []
            for start_frame, end_frame in intervals:
                # Get averaged embedding from start frame to end frame
                embeddings_per_layer = get_embedding_across_layers_audio(start_frame, end_frame, reps, model)
                embeddings_as_lists = [emb.tolist() for emb in embeddings_per_layer]
                embeddings_list.append(embeddings_as_lists)
            embeddings_audio[word] = embeddings_list  
    audio_name = audio.split('/')[-1]     
    output_dir = os.path.join(f'../experiments/{args.model}')
    output_file = os.path.join(output_dir, f'embeddings_words_{audio_name}.json')
    os.makedirs(output_dir, exist_ok=True)             
    save_json_with_embedding(output_file, audio_name, embeddings_audio)

In [10]:
32 // 16

2